## sentense from "test_Timescales of landscape response to divide migration .dock"

In [1]:
!pip install webdriver-manager

In [ ]:
!jupyter lab

In [13]:
from docx import Document
import os 
parent_path = r"C:\Users\miyar\OneDrive\デスクトップ\四年\文献演習"
fname = r"test_Timescales of landscape response to divide migration" + ".docx"
fpath = os.path.join(parent_path, fname)
docxfile = Document(fpath)
tables = docxfile.tables[0]
print("tables obj?: {}\ntype: {}".format(tables, type(tables)))
# dir(tables[0])
# columns = tables[0].columns
# print("columns obj?: {}\ntype: {}".format(columns, type(columns)))
# # cells = tables[0].column_cells()
# # print("cells obj?: {}\ntype: {}".format(cells, type(cells)))
# direction = tables[0].table_direction
# print("direction?: {}\ntype: {}".format(direction, type(direction)))
# for column in tables.columns:
#         column_text = []
#         for cell in column.cells:
#             text = cell.text
#             print(text)

row = tables.rows[4]
cell = row.cells[0]
para = cell.paragraphs[0]
for run in para.runs:
    if run.text:
        run.bold = True
        
docxfile.save(fpath)
# text = cell.text
# print("text: \n{}".format(text))
# print("dir cell: {}".format(dir(cell)))
# cells = tables.column_cells(1)
# cells[1].text

tables obj?: <docx.table.Table object at 0x0000024A4E930940>
type: <class 'docx.table.Table'>


In [47]:
import chromedriver_binary 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
from threading import Lock
import time
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--proxy-server="direct://"')
options.add_argument('--proxy-bypass-list=*')
options.add_argument('--start-maximized')
lock = Lock()
browser = webdriver.Chrome()
url = 'https://www.deepl.com/ja/translator'
browser.get(url)
# deepleにアクセスするまでしばらく待つ
time.sleep(1)
stextarea = browser.find_element_by_css_selector(
            '.lmt__textarea.lmt__source_textarea.lmt__textarea_base_style')
ttextarea = browser.find_element_by_css_selector(
    '.lmt__textarea.lmt__target_textarea.lmt__textarea_base_style')

lock.acquire()
stextarea.send_keys(text)
lock.release()

translated_text = ''

lock.acquire()
while not translated_text:
    time.sleep(2)
    translated_text = ttextarea.get_property('value')
#     translated_texts[i] = translated_text
# ここに翻訳結果が出力されたら成功（途中経過をjupyter notebookで確認する場合はコメントイン）
#print(translated_text)    
lock.release()

# sourse_textareaをクリアする
time.sleep(2)
stextarea.send_keys(Keys.CONTROL, "a")
stextarea.send_keys(Keys.BACKSPACE)

In [48]:
translated_text

'河川の侵食は水と土砂のフラックスによって支配されており、長期的な景観の進化における分水嶺の移動と排水の捕捉の重要性は古くから認識されている[e.g.,Gilbert, 1877;Davis, 1903;Glock, 1931;Bishop, 1995]。それにもかかわらず、岩石の隆起速度や気候条件の時空間的なパターンを復元することを目的とした現代の地形分析では、一般的に分水嶺の移動や排水の捕捉はほとんど複雑ではないと明示的または暗黙的に想定されてきた。この概念的枠組みは、排水ネットワークは強い擾乱を受けない限り、すぐに不変のパターンに安定するという、一世代前の景観進化モデルによって強化された[Tucker and Hancock, 2010]。分水嶺が安定しているか、河道プロファイルの応答のタイムスケールと比較してゆっくりと移動している場合、河道プロファイルの形態の解釈における主な課題は、河道斜面に対するテクトニック、気候、および岩石学的なコントロールを定量化し、これらの要因の空間的および時間的な変動を区別する必要があることに起因する[例えば、Wobusら、2006; Kirby and Whipple、2012; Whittaker、2012; Lague、2014によるレビューを参照]。岩石の特性と気候が最小限の空間的変動を示す場合、多くの研究が、岩石の隆起速度の空間的・時間的パターンがどのように景観形態、特に河川の縦断形態に現れるかについての洞察を深めることに貢献してきた[e.g. Kirby and Whipple 2001; Whittaker 2012; Lague 2014]。Kirby and Whipple, 2001, 2012; Lavé and Avouac, 2001; Duvall et al., 2004; Clark et al., 2005; Craddock et al., 2007; Harkins et al., 2007; Whittaker et al., 2007; Hilley and Arrowsmith, 2008; Cyr et al., 2010; Attal et al., 2011; Kirby and Ouimet, 2011; Miller et al., 2012; Whittaker, 2012;

In [49]:
cell_1 = row.cells[1]
cell_1.text = translated_text


In [50]:
fname = r"1_test_Timescales of landscape response to divide migration" + ".docx"
fpath = os.path.join(parent_path, fname)
docxfile.save(fpath)

In [51]:
import concurrent

# import automatic_translation.py

In [1]:
import automatic_translation as at
kwargs = {"Fpath": r"C:\Users\miyar\OneDrive\デスクトップ\四年\文献演習",
          "Fname": r"test_Timescales of landscape response to divide migration",
          "max_worker": 1}

sentenses = at.sentense_from_docx(**kwargs)


In [ ]:
import math 
max_worker = kwargs['max_worker']
n = math.ceil(len(sentenses)/max_worker)
split_sentenses = [sentenses[i:i+n] for i in range(0, len(sentenses), n)]
split_sentenses

In [4]:
sample_sentense = sentenses[:3]
print("len(sample_sentense): {}".format(len(sample_sentense)))
max_worker = kwargs['max_worker']
split_sentenses = at.split_sentenses(max_worker, sentenses=sample_sentense)
print(len(split_sentenses))

len(sample_sentense): 3
1


In [ ]:
sample_sentense

In [ ]:
translated_texts = at.translation_deepl(sourse_texts=sample_sentense)

In [ ]:
%%time
# sample_sentense = sentenses[:30]
translated_texts = at.run_multiThread(sentenses=sample_sentense, **kwargs)

In [ ]:
print("len(translated_texts) {}".format(len(translated_texts)))
print(translated_texts)
# at.sentense_to_docx(translated_texts, **kwargs)

In [8]:
import re
cant = 'これは、地形の勾配や深さが増すにつれて浸食速度が増加するためであり（Ahnert, 1970; DiBiase et al. 時間的に不変な形態としての定常状態または平衡ランドスケープの概念は、地形輸送法則の開発、パラメータ化、検証（Dietrich et al.2003; Kirkby, 1971）や、平衡形態からの逸脱に基づく過渡的なランドスケープの解釈（Kirby and Whipple, 2012; Tucker and Whipple, 2002; Whipple and Tucker, 1999）において中心的な役割を果たしてきた。平衡状態に到達する可能性は、境界条件の変化に対するランドスケープの応答時間に依存する（Howard, 1982; Whipple, 2001）。地殻変動や気候変動の擾乱後に定常状態に達するのに必要な時間は、一般的に、個々の流域が浸食と岩石の隆起の割合が等しくなるように河川の急勾配を調整する時間と同じであり、多くのランドスケープでは数百万年のオーダーであると考えられている（Pazzaglia, 2003; Whipple, 2001; Whipple and Tucker, 1999）。 '
can = re.search("\[...\]", cant)
type(can)

NoneType

In [8]:
at.sentense_to_docx(sample_sentense, translated_texts, **kwargs)
print(len(sample_sentense))
print(len(translated_texts))

3
3


In [10]:
import math 
l = [i for i in range(13)]
n = math.ceil(len(l)/3)
split = [l[i:i+n] for i in range(0, len(l), n)]
split

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12]]

In [11]:
l[10:15]

[10, 11, 12]

In [4]:
import re
s = "1abstract"
s_m = re.match("\d", s)
type(s_m)

re.Match

In [6]:
type(s_m) is re.Match

True

## 空白文字削除

In [5]:
import v3_automatic_translation as at
word_dict = {
            "ニックスゾーン": "ニックゾーン",
            }

kwargs = {"Fpath": r"E:\研究関連\文献演習", # 要約対象のwordファイルが格納されているディレクトリ
          "Fname": r"4(根幹モデル).Developing and exploring a theory for the.docx", # wordファイル名（拡張子込みで指定する）
          "max_worker": 4, # 同時実行スレッド数。各PC事に最大値が違うので要確認。
          "replace_dict" : word_dict,# 翻訳後に専門用語を正しく置換するための辞書(上記の辞書オブジェクト)
          "japaneseFont" : "游明朝 (本文のフォント - 日本語)", # 翻訳された日本語のフォントを指定（詳しいフォント名はwordを参照）
         } 
transltor = at.auto_translator(**kwargs)
transltor.insert_pt = 0
sentenses = transltor._sentence_from_docx()


Total sensentces in this articl = 1
Total sensentces in this articl = 61


In [6]:
for i in range(len(sentenses)):
    text = sentenses[i] 
    if text == "":
        print("i={} null!!!!!!!!!!!!!!!!!\n\n".format(i))
    # else:
    #     print("i={} {}".format(i, text))

In [ ]:
sentenses

In [4]:
transltor.eg_sentence

['The calculation of the radius of curvature along two stream segments in a raster grid with D8 flow routing presents a challenge, as the angle between segments is discretized; the two segments may form a straight line, in which case the angle is equal to 0◦ , form a 45◦ angle, or form a 90◦ angle. In order to reduce the impact of this discretization, we assume that each of these three cases represents a continuum of possible radii of curvature. Cases of two straight segments are treated as if the actual angle between them ranges anywhere between +22.5 and −22.5◦ . If one takes the average among these possible angles, the resulting inverse radius of curvature is 0.23/dx, where dx is the cell size in the flow direction. Similarly, we assume that a 45◦ bend represents a continuum of possible angles between the two segments ranging from 22.5–63.5◦ , resulting in an inverse radius of curvature of 0.67/dx. Following the same principle for a 90◦ bend gives a mean inverse radius of curvature 

In [9]:
from docx import Document
from docx.oxml.ns import qn
import os 
parent_path = r"E:\研究関連\文献リスト\knickpoint"
fname = r"デバック用" + ".docx"
fpath = os.path.join(parent_path, fname)
docxfile = Document(fpath)
tables = docxfile.tables[0]
o_cells = tables.column_cells(1)
runs = o_cells[1].paragraphs[0].runs
for run in runs:
    run.font.name = "游明朝 (本文のフォント - 日本語)"
    run._element.rPr.rFonts.set(qn('w:eastAsia'), run.font.name)
    print(run.text)
docxfile.save(fpath)
# print(o_cells[0].paragraphs[0].style.name)

地形学の主要な目的の一つは、地形が地殻変動、気候変動、侵食の履歴をどのように記録しているかを理解することである。この枠組みの中で、地形は一般的に定常状態と過渡状態に分類される[Kirby and Whipple, 2012]。定常的な地形とは、気候、岩石の強度、侵食プロセスによって決まる侵食速度と、隆起が丘陵斜面の低下と河川の侵食によって釣り合うように地形が調整されたものである。対照的に、過渡的な地形は、テクトニクスまたは侵食の条件に突然または持続的な変化が生じた場合に存在し、侵食がテクトニクスと釣り合わず、地形が徐々に調整される [Whipple and Tucker, 1999; Whipple, 2001; Tucker and Whipple, 2002; Kirby and Whipple, 2012; Whittaker, 2012]。定常的な地形では、侵食、隆起、侵食速度の間のバランスによって、傾斜、曲率、またはレリーフなどの地形指標が、気候、テクトニクス、および岩石の強度などの現在の環境条件と相関していることがわかる。過渡的な地形では、地形は徐々に調整され、地形指標は現在の環境条件に適応している地域と過去の環境条件に適応している「残存地形」との対比を明らかにすることができる。


In [ ]:
from docx import Document
from docx.oxml.ns import qn
import os 
parent_path = r"C:\Users\miyar\OneDrive\デスクトップ\研究関連"
fname = r"sample.docx"
fpath = os.path.join(parent_path, fname)
docxfile = Document(fpath)
tables = docxfile.tables[0]
o_cells = tables.column_cells(0)
for cell in o_cells:
    print(cell.text, "\n", cell.text == "")

# main

### 各セクションタイトル部分は先頭文字が数字になっているか確認＋woed内の各セルの末尾に余計な改行がないか確認

### automatic_translation.pyファイルの最後にあるif__name__=="__main__"以降のファイル名等を変更後に以下のセルを実行してください。

In [2]:
%run -i -t v4_automatic_translation.py

Total sensentces in this articl = 1
len(split_list): 1
i=0
split_list: ['Several mechanisms exist for generating ニックポイント in meandering channels (e.g., Frankel et al., 2007; Seidl and Dietrich, 1992). In addition, if ニックポイント are diffusive or stationary (Seidl and Dietrich, 1992), then strath terraces formed from our proposed mechanism might be limited to a discrete zone upstream of a cutoff, or perhaps not present at all. We therefore emphasize that our goal here is not to offer a comprehensive explanation for unpaired strath terraces along the Smith River (and elsewhere). Rather, our goal is to show how unpaired strath terraces can form entirely as an instability triggered by bedrock river meandering. We note that belts of uplifted clastic sedimentary rocks, where strath terraces and bedrock river meanders are commonly observed, compose wide swaths of most active mountain ranges on Earth. Given the instabilities that are inherent to these meandering channels, we therefore question the 

NameError: name 'WebDriverWait' is not defined


IPython CPU timings (estimated):
  User   :       2.06 s.
  System :       0.00 s.
Wall time:       7.89 s.


In [ ]:
def _translation_deepl(self, sourse_texts=[]):

        """
        抜き出した英文をdeeplにコピペして翻訳する関数。引数は翻訳する文章が要素のリスト

        """

        res = requests.get('https://chromedriver.storage.googleapis.com/LATEST_RELEASE')
        browser = webdriver.Chrome(ChromeDriverManager(res.text).install())
        url = 'https://www.deepl.com/ja/translator'
        browser.get(url)
        # deepleにアクセスするまでしばらく待つ
        time.sleep(1)
        # wordの文章をparagraph単位で翻訳していく
        # 先頭が数字の場合にセクションのタイトルと判定している。
        translated_texts = []
        for i in range(len(sourse_texts)):         
            sourse_text = sourse_texts[i]

            print("i={}\n: {}\n".format(i, sourse_text))

            stextarea = browser.find_element_by_css_selector(
                '.lmt__textarea.lmt__source_textarea.lmt__textarea_base_style')
            ttextarea = browser.find_element_by_css_selector(
                '.lmt__textarea.lmt__target_textarea.lmt__textarea_base_style')
            
            lock.acquire()
            # stextarea.send_keys(sourse_text) #この方法だと絵文字（数式を送る事ができない）
            # javascriptを仕込む方法に変更 2022/4/22 https://tech.bita.jp/article/19
            INPUT_EMOJI = """
            arguments[0].value += arguments[1];
            arguments[0].dispatchEvent(new Event('change'));
            """
            browser.execute_script(INPUT_EMOJI, stextarea, sourse_text)
            lock.release()

            translated_text = ''

            lock.acquire()
            # 完全に翻訳されるまで繰り返す。判定は翻訳後文章の文字数が0かどうか
            # 文字数が0の場合は繰り返される。
            sec = self.wating_sec
            while not translated_text:
                # print("keep now....")
                time.sleep(sec)
                translated_text = ttextarea.get_property('value')
                print("translated_text: \n{}\n len: {}\n ".format(translated_text, len(translated_text))) 
                can_translated = re.search("\[\.\.\.\]", translated_text)
                if type(can_translated) is re.Match:
                    print("\ntry again\n")
                    translated_text = ''
                    sec += 2
                elif len(translated_text) == 0: # 翻訳されていない場合（[...]すら表示されていないパターン）
                    print("\ntry again\n")
                    translated_text = ''
                    sec += 2
                else:
                    if self.replace_flag_jp_to_jp:
                        translated_texts.append(self._keyword_replace(translated_text))
                    else:
                        translated_texts.append(translated_text)

            # ここに翻訳結果が出力されたら成功（途中経過をjupyter notebookで確認する場合はコメントイン）
            #print(translated_text)    
            lock.release()

            # sourse_textareaをクリアする
            time.sleep(2)
            stextarea.send_keys(Keys.CONTROL, "a")
            stextarea.send_keys(Keys.BACKSPACE)
            
        return translated_texts

In [3]:
from bs4 import BeautifulSoup
import requests

sourse_text = "こんにちは"

url = "https://www.deepl.com/ja/translator"

data = {
    "text": sourse_text,
    "source_lang": "EN",
    "target_lang": "JA"
}

res = requests.post(url, data=data)
soup = BeautifulSoup(res.text, 'html.parser')

translated = soup.find("div", {"class":"lmt__translations_as_text"})

if translated:
    translated_text = translated.text
    print(translated_text)
else:
    print("Element not found")



Element not found


In [ ]:
#headlessui-tabs-panel-7 > div > div.border-dark-7.border.bg-white.shadow-sm.xl\:rounded-lg > section > div > div.rounded-bl-inherit.rounded-br-inherit.grid.grid-cols-1.grid-rows-\[auto_auto\].min-\[768px\]\:grid-cols-\[1fr_auto_1fr\].min-\[768px\]\:grid-rows-\[1fr\].TextTranslatorLayout-module--bothareas--wXSVy > div.rounded-bl-inherit.relative.z-\[1\].min-h-\[240px\].min-w-0.md\:min-h-\[clamp\(250px\,50vh\,557px\)\] > section > div > div.relative.flex-1 > d-textarea > div:nth-child(1)

## end point

In [3]:
print("translation")

translation


In [4]:
import re
translated_text = """ここで、z（次元の長さ。以下、次元は括弧内に表記し、[L]ength, [M]ass, [T]time と略す）は河道の標高、x [L] は縦座標、A [L2 ] は排水面積、U [L T1 ] は隆起率、K [T1 L(12m) ] は気候・岩相等の影響を包含したエロディビリティ係数である。指数 m と n は経験的に導かれた係数であるが、n の値は岩盤切開の力学に基づいて提案されており、2/3 から 7/3 の間である [Whipple et al.，2000]。
                    U と K は共に空間と時間の関数であり、K は土砂供給量の関数でもある [Sklar and Dietrich, 2004; Cowie et al., 2008; Hobley et al., 2011]、
                    侵食閾値と流路幅の調整の影響を取り込む [Snyder et al., 2003; Lague, 2010; Attal et al., 2011 など] ことができる。
                    ストリームパワーアプローチとその限界、および代替案については、Lague [2013]がレビューしている。
                    Perron and Royden [2013]によって考案された積分法（Royden and Perron [2013]も参照）は，
                    調査対象の流路の進化が式（1）で記述できることを前提としている．"""
can_translated = re.search("\[\.\.\.\]", translated_text)
print(type(can_translated) is re.Match)
print(can_translated.group())

False


AttributeError: 'NoneType' object has no attribute 'group'

In [7]:
word_dict = {
                "ニックスゾーン": "ニックゾーン",
                "ナイフゾーン": "ニックゾーン",
                "ニックスポイント": "ニックポイント",
                "キックポイント": "ニックポイント",
                "ナイフポイント": "ニックポイント",
                "ノッチポイント": "ニックポイント",
                "浸食": "侵食",
                "切り込み": "侵食",
                "侵食効率": "侵食速度", 
                "侵食率": "侵食速度",
                "基盤レベル":"ベースレベル",
                "基底レベル":"ベ-スレベル",
                "ランドスケープ": "地形",
                "景観": "地形",
                "風景": "地形",
                "岩石の隆起": "隆起",
                "岩盤の隆起": "隆起",
                "捕獲": "河川争奪",
                "ストリームキャプチャー": "河川争奪",
                "集水域": "流域",
                "排水面積": "流域面積",
                "盆地": "流域",
                "ディバイド" : "分水界",
                "分水嶺" : "分水界",
                "流力モデル":"ストリームパワーモデル",
                "います。" : "いる。",
                "排水路ネットワーク": "流路網",
                "排水ネットワーク": "流路網",
                }

In [10]:
sentence = "河川は通常、急勾配で凸状のニックスゾーン区間内で浸食を局限することによって、水位低下に対応する。局所的な侵食は、ニックスゾーン区間を上流に移動させる。このような移動するニックスゾーンは、テクトニクスや侵食効率の変化に対する地形の反応のペースを決定し、基盤レベルの低下のタイミングと原因を定量化するのに役立つ。ニックスゾーンの同定には、通常、急勾配の区間を個別に選択する必要がある。このプロセスは面倒で主観的であり、ニックスゾーンのサイズを測定する効率的な手段もない。我々は、χ空間（流域面積の正規化）の枠組みでニックゾーン区間の境界を選択することにより、この手順を自動化するアルゴリズムを構築した。自動化機能により、ユーザーが選んだニックゾーンのサブセットに対してアルゴリズムのパラメータを較正する。アルゴリズムは、これらのパラメータを一貫した基準として使用し、ニックスゾーンを客観的に特定し、各ニックスゾーンリーチの高さ、長さ、および傾斜を測定する。我々は、南カリフォルニアのサンタクルーズ島にある6つの集水域（幹流の長さ：2.1～5.4km）の1、10、30m解像度のデジタル標高モデル（DEM）を用いてアルゴリズムをテストした。1 m DEMでは、アルゴリズムで選択されたニックゾーンは、手作業で選択されたニックゾーン（n = 178）の93%（100 m以下の空間精度）、88%（50 m以内の精度）、46%（10 m以内の精度）を確認しました。10 mと30 m DEMでは、同様の精度（100 m以下で88-86%、50 m以下で82%）でした（それぞれn = 38 と36）。本アルゴリズムにより、地質構造、地図上の地形、丘陵斜面の形態とニックスゾーンのサイズと位置を効率的に地域比較することができ、過渡的なランドスケープのダイナミクスを特徴付けるアプローチを促進することができる。"
replace_dict = {'ニックスゾーン': "ニックゾーン",
                "浸食": "侵食",
                "侵食効率": "侵食速度", 
                "基盤レベル": "ベースレベル",
                "ランドスケープ": "地形",
                }
index_list = []
for i, item in enumerate(replace_dict.items()):
    sentence = sentence.replace(item[0], '{'+str(i)+'}')
    index_list.append(item[1])
print('\n', sentence)
sentence = sentence.format(*index_list)
print('\n', sentence)


 河川は通常、急勾配で凸状の{0}区間内で{1}を局限することによって、水位低下に対応する。局所的な侵食は、{0}区間を上流に移動させる。このような移動する{0}は、テクトニクスや{2}の変化に対する地形の反応のペースを決定し、{3}の低下のタイミングと原因を定量化するのに役立つ。{0}の同定には、通常、急勾配の区間を個別に選択する必要がある。このプロセスは面倒で主観的であり、{0}のサイズを測定する効率的な手段もない。我々は、χ空間（流域面積の正規化）の枠組みでニックゾーン区間の境界を選択することにより、この手順を自動化するアルゴリズムを構築した。自動化機能により、ユーザーが選んだニックゾーンのサブセットに対してアルゴリズムのパラメータを較正する。アルゴリズムは、これらのパラメータを一貫した基準として使用し、{0}を客観的に特定し、各{0}リーチの高さ、長さ、および傾斜を測定する。我々は、南カリフォルニアのサンタクルーズ島にある6つの集水域（幹流の長さ：2.1～5.4km）の1、10、30m解像度のデジタル標高モデル（DEM）を用いてアルゴリズムをテストした。1 m DEMでは、アルゴリズムで選択されたニックゾーンは、手作業で選択されたニックゾーン（n = 178）の93%（100 m以下の空間精度）、88%（50 m以内の精度）、46%（10 m以内の精度）を確認しました。10 mと30 m DEMでは、同様の精度（100 m以下で88-86%、50 m以下で82%）でした（それぞれn = 38 と36）。本アルゴリズムにより、地質構造、地図上の地形、丘陵斜面の形態と{0}のサイズと位置を効率的に地域比較することができ、過渡的な{4}のダイナミクスを特徴付けるアプローチを促進することができる。

 河川は通常、急勾配で凸状のニックゾーン区間内で侵食を局限することによって、水位低下に対応する。局所的な侵食は、ニックゾーン区間を上流に移動させる。このような移動するニックゾーンは、テクトニクスや侵食速度の変化に対する地形の反応のペースを決定し、ベースレベルの低下のタイミングと原因を定量化するのに役立つ。ニックゾーンの同定には、通常、急勾配の区間を個別に選択する必要がある。このプロセスは面倒で主観的であり、ニックゾーンのサイズを測定する効率的な手段もない。我々は、χ空

In [12]:
print(type(replace_dict) == dict)

True
